In [1]:
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt

In [2]:
test = scio.loadmat('dataset/test.mat')
train = scio.loadmat('dataset/train.mat')
test_x = np.array(test['X'])
test_y = np.array(test['y'])
train_x = np.array(train['X'])
train_y = np.array(train['y'])

test_x = np.transpose(test_x, (3, 0, 1, 2))
train_x = np.transpose(train_x, (3, 0, 1, 2))

train_x = train_x.astype(float)
test_x = test_x.astype(float)
test_x.shape, test_y.shape, train_x.shape, train_y.shape

((26032, 32, 32, 3), (26032, 1), (73257, 32, 32, 3), (73257, 1))

In [3]:
for i in range(len(test_y)):
    if test_y[i] == 10:
        test_y[i] = 0
for i in range(len(train_y)):
    if train_y[i] == 10:
        train_y[i] = 0

In [4]:
def rgb_to_gray(rgb_image):
    gray_w = np.array([0.299, 0.587, 0.114]).reshape(3,1)
    gray_image = np.matmul(rgb_image, gray_w)
    return gray_image

In [5]:
train_x_gray = np.zeros((train_x.shape[0], 32, 32, 1))
test_x_gray = np.zeros((test_x.shape[0], 32, 32, 1))

for i in range(train_x.shape[0]):
    train_x_gray[i] = rgb_to_gray(train_x[i])

for i in range(test_x.shape[0]):
    test_x_gray[i] = rgb_to_gray(test_x[i])

(73257, 32, 32)

In [8]:
def min_max_scaling(image_data):
    flattened_data = image_data.reshape((-1, 2))
    min_vals = np.min(flattened_data, axis=0)
    max_vals = np.max(flattened_data, axis=0)

    scaled_data = (flattened_data - min_vals) / (max_vals - min_vals)

    scaled_image_data = scaled_data.reshape(image_data.shape)
    return scaled_image_data

In [9]:
for i in range(train_x_gray.shape[0]):
    train_x_gray[i] = min_max_scaling(train_x_gray[i])
for i in range(test_x_gray.shape[0]):
    test_x_gray[i] = min_max_scaling(test_x_gray[i])

In [10]:
class Relu():
    def __init__(self) -> None:
        pass

    def forward(self, input):
        self.input = input
        self.output = np.where(self.input <= 0, 0, input)
        return self.output

    def backprop(self, theta):
        d = np.where(self.input > 0, 1, 0)
        return d * theta

In [11]:
class AdamOptimizer:
    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = [np.zeros_like(param) for param in params]
        self.v = [np.zeros_like(param) for param in params]
        self.t = 0
        self.params = params  

    def update(self, grads):
        self.t += 1
        for i, grad in enumerate(grads):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grad ** 2)
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            update = self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            self.params[i] -= update  

In [12]:
class Conv3x3:
    def __init__(self, c, num_filters) -> None:
        self.input_c = c
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3, self.input_c) * np.sqrt(2 / (3 * 3 * self.input_c))


    def img_iterator(self, image):
        h, w, c = image.shape
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3), :]
                yield i, j, im_region


    def forward(self, input):
        self.input = input
        self.input_h, self.input_w, self.input_c = self.input.shape
        self.output = np.zeros((self.input_h - 2, self.input_w - 2, self.num_filters))

        for i, j, img in self.img_iterator(input):
            # img(3, 3, c) * filters(n, 3, 3, c) = (n, 3, 3, c)
            self.output[i, j] = np.sum(img * self.filters, axis=(1, 2, 3))
        return self.output


    def backprop(self, next_theta):
        theta = np.zeros((self.input_h, self.input_w, self.input_c))
        delta = np.zeros(self.filters.shape)

        for i, j, img in self.img_iterator(self.input):
            for f in range(self.num_filters):
                delta[f] += next_theta[i, j, f] * img
        
        for f in range(self.num_filters):
            for i in range(self.input_h - 2):
                for j in range(self.input_w - 2):
                    theta[i:(i + 3), j:(j + 3)] += next_theta[i, j, f] * self.filters[f]          
        # self.filters -= lr * delta
        return theta, delta

In [13]:
# 2*2的最大值池化
class MaxPool:
    def __init__(self) -> None:
        pass


    def img_iterator(self, image):
        input_h, input_w, input_c = image.shape
        output_h, output_w = input_h//2, input_w//2

        for i in range(output_h):
            for j in range(output_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield i, j, im_region
    

    def forward(self, input):
        self.input = input
        self.input_h, self.input_w, self.input_c = self.input.shape
        self.output_h, self.output_w = self.input_h//2, self.input_w//2
        self.output = np.zeros((self.output_h, self.output_w, self.input_c))

        for i, j, img in self.img_iterator(self.input):
            self.output[i, j] = np.max(img, axis=(0, 1))      
        return self.output
    

    def backprop(self, next_theta):
        theta = np.zeros((self.input_h, self.input_w, self.input_c))
        for i, j, img in self.img_iterator(self.input):
            h, w, c = img.shape   
            a_max = np.max(img, axis=(0, 1))
            
            for i2 in range(h):
                for j2 in range(w):
                    for c2 in range(c):
                        if img[i2, j2, c2] == a_max[c2]:
                            theta[i * 2 + i2, j * 2 + j2, c2] = next_theta[i, j, c2]
        return theta

In [14]:
class Dense:
    def __init__(self, input_size, output_size) -> None:
        self.w = np.random.randn(output_size, input_size) * np.sqrt(2 / input_size)


    def forward(self, input):
        self.input = input 
        self.output = np.matmul(self.w, self.input) 
        return self.output
    
    
    def backprop(self, next_theta):
        theta = np.dot(self.w.T, next_theta)
        delta = np.outer(next_theta, self.input)
        # self.w = self.w - lr * delta
        return theta, delta

In [15]:
class Softmax:
    def __init__(self) -> None:
        pass
        
        
    def forward(self, input): 
        input_exp = np.exp(input)
        self.output = input_exp / np.sum(input_exp, axis=0)
        return self.output
    
    
    def getloss(self, label):
        self.label_onehot = np.zeros_like(self.output)              
        self.label_onehot[label] = 1
        loss = -np.sum(np.log(self.output) * self.label_onehot)
        return loss


    def backprop(self):
        theta = self.output - self.label_onehot
        return theta

In [16]:
conv1 = Conv3x3(1, 32) # -> 30*30*32
relu1 = Relu()
pool1 = MaxPool()   # -> 15*15*32
conv2 = Conv3x3(32, 64) # -> 13*13*64
relu2 = Relu()
conv3 = Conv3x3(64, 64) # -> 11*11*64
relu3 = Relu()
pool3 = MaxPool()   # -> 5*5*64

dense1 = Dense(1600, 512)
relu4 = Relu()
dense2 = Dense(512, 10)

softmax = Softmax()

In [19]:
adam_optimizer = AdamOptimizer(params=[conv1.filters, conv2.filters, conv3.filters, dense1.w, dense2.w])

In [17]:
def forward(img, label):
    a = conv1.forward(img)
    a = relu1.forward(a)
    a = pool1.forward(a)
    a = conv2.forward(a)
    a = relu2.forward(a)
    a = conv3.forward(a)
    a = relu3.forward(a)
    a = pool3.forward(a)
    
    a = a.reshape(-1)

    a = dense1.forward(a)
    a = relu4.forward(a)
    a = dense2.forward(a)

    out = softmax.forward(a)

    loss = softmax.getloss(label)
    ans = np.argmax(out)
    acc = 1 if ans == label else 0
    return out, loss, acc

In [18]:
def train(img, label):
    out, loss, acc = forward(img, label)
 
    # 反向传播
    theta = softmax.backprop() 
 
    theta, dense2_grad = dense2.backprop(theta)
    theta = relu4.backprop(theta) 
    theta, dense1_grad = dense1.backprop(theta)

    theta = theta.reshape(5,5,64)

    theta = pool3.backprop(theta)
    theta = relu3.backprop(theta) 
    theta, conv3_grad = conv3.backprop(theta)
    theta = relu2.backprop(theta) 
    theta, conv2_grad = conv2.backprop(theta)
    theta = pool1.backprop(theta)
    theta = relu1.backprop(theta) 
    theta, conv1_grad = conv1.backprop(theta)
    adam_optimizer.update([conv1_grad, conv2_grad, conv3_grad, dense1_grad, dense2_grad])
    return loss, acc

In [20]:
best_accuracy = 0
best_params = [conv1.filters.copy(), conv2.filters.copy(), conv3.filters.copy(), dense1.w.copy(), dense2.w.copy()]

In [21]:
for epoch in range(1):
    print('--- Epoch %d ---' % (epoch + 1))
    loss = 0
    num_correct = 0

    for i, (img, label) in enumerate(zip(train_x_gray, train_y)):
        if i > 0 and i % 1000 == 999:
            print(f'[Step {i + 1}] Past 1000 steps: Average Loss {loss / 1000} | Accuracy: {num_correct / 1000 :.2%}')
            
            current_accuracy = num_correct/1000
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_params = [conv1.filters.copy(), conv2.filters.copy(), conv3.filters.copy(), dense1.w.copy(), dense2.w.copy()]
            
            loss = 0
            num_correct = 0

        l, acc = train(img, label)
        loss += l
        num_correct += acc

conv1.filters, conv2.filters, conv3.filters, dense1.w, dense2.w = best_params

--- Epoch 1 ---
[Step 1000] Past 1000 steps: Average Loss 2.0710472271963782 | Accuracy: 29.30%
[Step 2000] Past 1000 steps: Average Loss 1.2336045383356316 | Accuracy: 59.70%
[Step 3000] Past 1000 steps: Average Loss 0.9870197622522251 | Accuracy: 70.60%
[Step 4000] Past 1000 steps: Average Loss 0.8287701501933068 | Accuracy: 74.90%
[Step 5000] Past 1000 steps: Average Loss 0.8730084303513184 | Accuracy: 72.30%
[Step 6000] Past 1000 steps: Average Loss 0.8392607043791046 | Accuracy: 75.10%
[Step 7000] Past 1000 steps: Average Loss 0.7712740253156015 | Accuracy: 77.00%
[Step 8000] Past 1000 steps: Average Loss 0.647756526392497 | Accuracy: 79.10%
[Step 9000] Past 1000 steps: Average Loss 0.7028852825737771 | Accuracy: 78.50%
[Step 10000] Past 1000 steps: Average Loss 0.696733508193185 | Accuracy: 78.60%
[Step 11000] Past 1000 steps: Average Loss 0.5849927179708961 | Accuracy: 81.60%
[Step 12000] Past 1000 steps: Average Loss 0.7163546743914582 | Accuracy: 79.10%
[Step 13000] Past 1000 

In [ ]:
import pickle
with open('model_params.pkl', 'wb') as file:
    pickle.dump(best_params, file)

In [22]:
loss = 0
num_correct = 0
for img, label in zip(test_x_gray, test_y):
    _, l, acc = forward(img, label)
    loss += l
    num_correct += acc
 
num_tests = len(test_x)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)


Test Loss: 0.4972428456464234
Test Accuracy: 0.8567916410571604
